<h3> Listing data files </h3>

In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path,name,size
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008


<h3> Defining Schema</h3>

In [0]:
from pyspark.sql.types import *

from pyspark.sql.functions import *

#Reference to the path where input data is stored
inputPath = "/databricks-datasets/structured-streaming/events/"

jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

<h3> Initializing the stream </h3>

In [0]:
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

In [0]:
# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# To check if the DF is actually a streaming DF
streamingCountsDF.isStreaming

Out[7]: True

<h3>Starting the Streaming Job</h3>

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [0]:
# Blocks until all available data in the source has been processed and committed to the sink. This method is intended for testing.
query.processAllAvailable() 

# Prints the (logical and physical) plans to the console for debugging purpose.
query.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@3bd03597
+- *(4) HashAggregate(keys=[action#3352, window#1061], functions=[finalmerge_count(merge count#1106L) AS count(1)#1059L])
 +- StateStoreSave [action#3352, window#1061], state info [ checkpoint = dbfs:/local_disk0/tmp/temporary-193e207a-7b25-4947-bbe0-d06431da334c/state, runId = 30fcc1de-a4a6-48ff-a503-b3104d29eed9, opId = 0, ver = 49, numPartitions = 2], Complete, 0, 2
 +- *(3) HashAggregate(keys=[action#3352, window#1061], functions=[merge_count(merge count#1106L) AS count#1106L])
 +- StateStoreRestore [action#3352, window#1061], state info [ checkpoint = dbfs:/local_disk0/tmp/temporary-193e207a-7b25-4947-bbe0-d06431da334c/state, runId = 30fcc1de-a4a6-48ff-a503-b3104d29eed9, opId = 0, ver = 49, numPartitions = 2], 2
 +- *(2) HashAggregate(keys=[action#3352, window#1061], functions=[merge_count(merge count#1106L) AS count#1106L])
 +- Exchange hashpartitioning(action#3352, window#1061, 2), true, [id=#7813]
 +- *(1) HashAggregate(keys=[action#3352, window#1061], functions=[partial_count(1) AS count#1106L])
 +- *(1) Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) as double) = (cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) THEN (CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) + 1) ELSE CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) END + 0) - 1) * 3600000000) + 0), LongType, TimestampType), end, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) as double) = (cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) THEN (CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) + 1) ELSE CEIL((cast((precisetimestampconversion(time#3351, TimestampType, LongType) - 0) as double) / 3.6E9)) END + 0) - 1) * 3600000000) + 3600000000), LongType, TimestampType)) AS window#1061, action#3352]
 +- *(1) Filter isnotnull(time#3351)
 +- FileScan json [time#3351,action#3352] Batched: false, DataFilters: [isnotnull(time#3351)], Format: JSON, Location: InMemoryFileIndex[dbfs:/databricks-datasets/structured-streaming/events/file-49.json], PartitionFilters: [], PushedFilters: [IsNotNull(time)], ReadSchema: struct<time:timestamp,action:string>

In [0]:
# Returns the StreamingQueryException if the query was terminated by an exception, or None.
query.exception()

<h3> Properties of a query </h3>

In [0]:
# Returns the unique id of this query that persists across restarts from checkpoint data. 
print("id: " , query.id)

# Return boolean: whether this streaming query is currently active or not.
print("isActive: " , query.isActive)

# Returns the user-specified name of the query, or null if not specified.
print("name: " , query.name)

# Returns the unique id of this query that does not persist across restarts. 
print("runId: " , query.runId)

# Returns the current status of the query.
print("status: ", query.status)

id: c861eaba-9a45-4a97-8bd4-35687ae4365f
isActive: False
name: counts
runId: 30fcc1de-a4a6-48ff-a503-b3104d29eed9
status: {'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

In [0]:
# Returns the most recent StreamingQueryProgress update of this streaming query or None if there were no progress updates.
print("lastProgress:", query.lastProgress)

lastProgress: {'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:26:29.327Z', 'batchId': 49, 'numInputRows': 2000, 'inputRowsPerSecond': 674.308833445718, 'processedRowsPerSecond': 614.8170919151552, 'durationMs': {'addBatch': 830, 'getBatch': 128, 'latestOffset': 1301, 'queryPlanning': 103, 'triggerExecution': 3252, 'walCommit': 444}, 'stateOperators': [{'numRowsTotal': 104, 'numRowsUpdated': 4, 'memoryUsedBytes': 30736, 'customMetrics': {'loadedMapCacheHitCount': 196, 'loadedMapCacheMissCount': 0, 'stateOnCurrentVersionSizeBytes': 24672}}], 'sources': [{'description': 'FileStreamSource[dbfs:/databricks-datasets/structured-streaming/events]', 'startOffset': {'logOffset': 48}, 'endOffset': {'logOffset': 49}, 'numInputRows': 2000, 'inputRowsPerSecond': 674.308833445718, 'processedRowsPerSecond': 614.8170919151552}], 'sink': {'description': 'MemorySink', 'numOutputRows': 104}}

In [0]:
# Returns an array of the most recent [[StreamingQueryProgress]] updates for this query.
print("recentProgress:", query.recentProgress)

recentProgress: [{'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:24:03.397Z', 'batchId': 0, 'numInputRows': 2000, 'processedRowsPerSecond': 511.9017148707448, 'durationMs': {'addBatch': 1759, 'getBatch': 76, 'latestOffset': 835, 'queryPlanning': 86, 'triggerExecution': 3906, 'walCommit': 480}, 'stateOperators': [{'numRowsTotal': 6, 'numRowsUpdated': 6, 'memoryUsedBytes': 1872, 'customMetrics': {'loadedMapCacheHitCount': 0, 'loadedMapCacheMissCount': 0, 'stateOnCurrentVersionSizeBytes': 1584}}], 'sources': [{'description': 'FileStreamSource[dbfs:/databricks-datasets/structured-streaming/events]', 'startOffset': None, 'endOffset': {'logOffset': 0}, 'numInputRows': 2000, 'processedRowsPerSecond': 511.9017148707448}], 'sink': {'description': 'MemorySink', 'numOutputRows': 6}}, {'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:24:07.308Z', 'batchId': 1, 'numInputRows': 2000, 'inputRowsPerSecond': 511.3781641523907, 'processedRowsPerSecond': 647.6683937823834, 'durationMs': {'addBatch': 1073, 'getBatch': 119, 'latestOffset': 694, 'queryPlanning': 44, 'triggerExecution': 3088, 'walCommit': 559}, 'stateOperators': [{'numRowsTotal': 8, 'numRowsUpdated': 4, 'memoryUsedBytes': 3056, 'customMetrics': {'loadedMapCacheHitCount': 4, 'loadedMapCacheMissCount': 0, 'stateOnCurrentVersionSizeBytes': 1984}}], 'sources': [{'description': 'FileStreamSource[dbfs:/databricks-datasets/structured-streaming/events]', 'startOffset': {'logOffset': 0}, 'endOffset': {'logOffset': 1}, 'numInputRows': 2000, 'inputRowsPerSecond': 511.3781641523907, 'processedRowsPerSecond': 647.6683937823834}], 'sink': {'description': 'MemorySink', 'numOutputRows': 8}}, {'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:24:10.406Z', 'batchId': 2, 'numInputRows': 2000, 'inputRowsPerSecond': 645.577792123951, 'processedRowsPerSecond': 468.05523051720104, 'durationMs': {'addBatch': 1648, 'getBatch': 155, 'latestOffset': 687, 'queryPlanning': 145, 'triggerExecution': 4273, 'walCommit': 1185}, 'stateOperators': [{'numRowsTotal': 10, 'numRowsUpdated': 4, 'memoryUsedBytes': 3552, 'customMetrics': {'loadedMapCacheHitCount': 8, 'loadedMapCacheMissCount': 0, 'stateOnCurrentVersionSizeBytes': 2448}}], 'sources': [{'description': 'FileStreamSource[dbfs:/databricks-datasets/structured-streaming/events]', 'startOffset': {'logOffset': 1}, 'endOffset': {'logOffset': 2}, 'numInputRows': 2000, 'inputRowsPerSecond': 645.577792123951, 'processedRowsPerSecond': 468.05523051720104}], 'sink': {'description': 'MemorySink', 'numOutputRows': 10}}, {'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:24:14.681Z', 'batchId': 3, 'numInputRows': 2000, 'inputRowsPerSecond': 467.83625730994146, 'processedRowsPerSecond': 662.6905235255136, 'durationMs': {'addBatch': 1111, 'getBatch': 110, 'latestOffset': 538, 'queryPlanning': 95, 'triggerExecution': 3018, 'walCommit': 641}, 'stateOperators': [{'numRowsTotal': 12, 'numRowsUpdated': 4, 'memoryUsedBytes': 4144, 'customMetrics': {'loadedMapCacheHitCount': 12, 'loadedMapCacheMissCount': 0, 'stateOnCurrentVersionSizeBytes': 2944}}], 'sources': [{'description': 'FileStreamSource[dbfs:/databricks-datasets/structured-streaming/events]', 'startOffset': {'logOffset': 2}, 'endOffset': {'logOffset': 3}, 'numInputRows': 2000, 'inputRowsPerSecond': 467.83625730994146, 'processedRowsPerSecond': 662.6905235255136}], 'sink': {'description': 'MemorySink', 'numOutputRows': 12}}, {'id': 'c861eaba-9a45-4a97-8bd4-35687ae4365f', 'runId': '30fcc1de-a4a6-48ff-a503-b3104d29eed9', 'name': 'counts', 'timestamp': '2020-11-09T14:24:17.701Z', 'batchId': 4, 'numInputRows': 2000, 'inputRowsPerSecond': 662.2516556291391, 'processedRow

<h3> Streaming Query Manager </h3>

In [0]:
# Returns a list of active queries associated with this SQLContext
print("active: ", spark.streams.active)

# Returns an active query from this SQLContext or throws exception if an active query with this name doesn’t exist.
print("get active query:" , spark.streams.get(query.id))

# Forget about past terminated queries so that awaitAnyTermination() can be used again to wait for new terminations.
print("resetTerminated:" , spark.streams.resetTerminated())

active: []
get active query: <pyspark.sql.streaming.StreamingQuery object at 0x7f9a8e9fda50>
resetTerminated: None

<h3>Data Stream Writer</h3>

In [0]:
query.stop()

In [0]:
# foreach(f)
# Sets the output of the streaming query to be processed using the provided writer f. This is often used to write the output of a streaming query to arbitrary storage systems. 

#Using a function
def print_row(row):
    print(row)

writerFunc = streamingCountsDF.writeStream.foreach(print_row)

writerFunc

Out[41]: <pyspark.sql.streaming.DataStreamWriter at 0x7f9a8eb1b190>

In [0]:
# Print every row using a object with process() method
class RowPrinter:
    def open(self, partition_id, epoch_id):
        print("Opened %d, %d" % (partition_id, epoch_id))
        return True
    def process(self, row):
        print(row)
    def close(self, error):
        print("Closed with error: %s" % str(error))

writerObj = streamingCountsDF.writeStream.foreach(RowPrinter())

writerObj

Out[42]: <pyspark.sql.streaming.DataStreamWriter at 0x7f9a8dde2790>

In [0]:
# foreachBatch(func)
# Sets the output of the streaming query to be processed using the provided function. This is supported only the in the micro-batch execution modes (that is, when the trigger is not continuous).

def func(batch_df, batch_id):
    batch_df.collect()

writerBatch = streamingCountsDF.writeStream.foreachBatch(func)

writerBatch

Out[44]: <pyspark.sql.streaming.DataStreamWriter at 0x7f9a8e9efa50>

In [0]:
# trigger
# Set the trigger for the stream query. If this is not set it will run the query as fast as possible, which is equivalent to setting the trigger to processingTime='0 seconds'.

# trigger the query for execution every 5 seconds
writerP5 = streamingCountsDF.writeStream.trigger(processingTime='5 seconds')

# trigger the query for just once batch of data
writerOnce = streamingCountsDF.writeStream.trigger(once=True)

# trigger the query for execution every 5 seconds
writerC5 = streamingCountsDF.writeStream.trigger(continuous='5 seconds')